In [53]:
import pandas as pd

df_unbalanced = pd.read_csv('data.csv')

df = df_unbalanced.groupby('label', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 900), random_state=19)
).reset_index(drop=True)  # Reset index to remove unwanted artifacts

# Explicitly keep only the original columns to ensure structure is intact
df = df[df_unbalanced.columns]


/tmp/ipykernel_257611/4082056144.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df_unbalanced.groupby('label', group_keys=False).apply(


In [54]:
X = df['text']
y = df['label']
len(X)

2996

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)  # 80% training and 20% test
len(X_train)

2396

In [56]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

In [57]:
pipeMNB = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
pipeCNB = Pipeline([('tfidf', TfidfVectorizer()), ('clf', ComplementNB())])
pipeSVC = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [58]:
pipeMNB.fit(X_train, y_train)
predictMNB = pipeMNB.predict(X_test)
accuracy_score(y_test, predictMNB)

0.8383333333333334

In [61]:
# userText = """While working together at Plaid, our founders Julianna and Reed witnessed the security and user-experience shortcomings of dated authentication methods. They saw firsthand the endless frustration experienced by teams — engineers wasting valuable time building authentication solutions instead of focusing on their own product and users suffering the pain of clunky authentication experiences.
#
# To solve this, they set out to build an all-in-one platform for modern authentication. A full suite of passwordless solutions and password-based auth, rebooted for the modern age."""

userText = """Methods and systems for updating a calendar entry for an accommodation listing are disclosed. In one embodiment, the method comprises generating an availability model and an acceptance model for an accommodation listing in an accommodation reservation system and determining based on those models the probability that the accommodation listing would be able to be booked. Furthermore, the result of an accommodation search query can be filtered and/or sorted using the determined probability of booking.”"""
prediction = pipeMNB.predict([userText])
print(prediction)

['wipo']
